# Simulation Results

In [ ]:
library(ggplot2)
library(tidyr)
library(dplyr)
library(viridis)

# 定义自定义标签函数
custom_labels <- function(tau_values) {
  labels <- sapply(tau_values, function(x) paste("SNR =", x))
  return(labels)
}

In [ ]:
myPlot <- function(str, prefix, str2, prefix2) {
    res <- matrix(NA, nrow = 2250, ncol = 7)
    combs <- expand.grid(tau = rep(c(0.10, 0.25, 0.3, 0.5, 1.0), each = 50), n = c(500, 750, 1000), quantile = c(0.05, 0.25, 0.5))

    for (uu in 1 : 2250) {
        tmp <- paste0(prefix, str, "/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
            for (i in 1 : 7) {
                res[uu, i] <- temp$err[i + 1] # TODO: log scale
            }
            tmp1 <- paste0(prefix2, str2, "/Res_", uu, ".RDS")
            if (file.exists(tmp1)) {
                temp1 <- readRDS(tmp1)
                res[uu, 5] <- temp1$err[2]
            }
        } else {
            cat(uu, ", ", sep = "")
        }
    }

    res <- cbind(combs, res)
    res <- na.omit(res)

    res_df <- as.data.frame(res)
    colnames(res_df) <- c('tau', 'n', 'q', 'Uni', 'Ridge', 'Elastic', 'Smooth', 'Concave', 'FPCA', 'FPCA-R') 

    res_long <- pivot_longer(res_df, cols = c('Uni', 'Ridge', 'Elastic', 'Concave', 'FPCA', 'FPCA-R'), names_to = "Method", values_to = "value")

    # 指定 Method 的顺序
    res_long$Method <- factor(res_long$Method, levels = c("Elastic", "Ridge", "FPCA", "FPCA-R", "Uni", "Concave"))

    tau_values <- c(0.10, 0.25, 0.5)
    # n_values <- c(500, 750, 1000, 1500, 2000)
    n_values <- c(500, 750, 1000)
    q_values <- c(0.25)
    
    # 过滤出包含所需 tau 值的数据
    res_tau_filtered <- filter(res_long, tau %in% tau_values)
    res_n_filtered <- filter(res_tau_filtered, n %in% n_values)
    res_q_filtered <- filter(res_n_filtered, q %in% q_values)

    # 使用 ggplot 创建箱线图，并使用 facet_wrap 进行分面
    p <- ggplot(res_q_filtered, aes(x = factor(n), y = value, fill = Method)) +
        geom_boxplot(width = 0.7, outlier.size = 0.1, notch = TRUE) +
        facet_wrap(~ tau, nrow = 2, ncol = 3, labeller = labeller(tau = custom_labels)) +
        xlab(expression(n)) +
        ylab("Estimation Error") +
        theme_bw() +
        scale_fill_viridis(discrete = TRUE, guide = 'none', option = 'C') +
        theme(plot.title = element_text(hjust = 0.5, size = 20),
            axis.text = element_text(size = 15),
            axis.title = element_text(size = 20),
            legend.position = "bottom",
            strip.text = element_text(size = 15)) # + guides(fill = guide_legend()) 

    # 保存图像 # ? set height = 4 for plot without legend
    ggsave(paste0("figs/new-", str, ".pdf"), plot = p, width = 15, height = 4, units = "in")
}

## Estimation Errors
### With normal noise (Figures in section 6.3)

In [ ]:
prefix <- "Results-v01s/"
prefix2 <- "Results-v02s/"

str <- "01A-normal"
str2 <- "02A-normal"
myPlot(str, prefix, str2, prefix2)

str <- "01B-normal"
str2 <- "02B-normal"
myPlot(str, prefix, str2, prefix2)

str <- "01C-normal"
str2 <- "02C-normal"
myPlot(str, prefix, str2, prefix2)

### With t-distritbution noise (Figures in section 6.3)

In [ ]:
prefix <- "Results-v11s/"
prefix2 <- "Results-v13s/"

str <- "01A-t"
str2 <- "02A-t"
myPlot(str, prefix, str2, prefix2)

str <- "01B-t"
str2 <- "02B-t"
myPlot(str, prefix, str2, prefix2)

str <- "01B-t"
str2 <- "02B-t"
myPlot(str, prefix, str2, prefix2)

## Estimation Curves

In [27]:
combs <- expand.grid(tau = rep(c(0.10, 0.25, 0.3, 0.5, 1.0), each = 50), n = c(500, 750, 1000), quantile = c(0.05, 0.25, 0.5))

pdf("figs/11-3-curve-1.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 3, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 3, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 3, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 3, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 3)
}
dev.off()

png 
  2

In [28]:
# Spline with Ridge Penalty
pdf("figs/11-3-curve-2.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 4, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 4, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [29]:
pdf("figs/11-3-curve-3.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v13s/13-3/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 3, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 3, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = "orange", lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v13s/13-3/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = "orange", lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 3)
}
dev.off()

png 
  2

In [30]:
combs <- expand.grid(tau = rep(c(0.10, 0.25, 0.3, 0.5, 1.0), each = 50), n = c(500, 750, 1000), quantile = c(0.05, 0.25, 0.5))

pdf("figs/11-3s-curve-1.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3s/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 3, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3s/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 3, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [31]:
# ! ---------------------------
# ! Spline with Ridge Penalty
# ! ---------------------------
pdf("figs/11-3s-curve-2.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3s/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 4, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3s/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 4, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [32]:
pdf("figs/11-3s-curve-3.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v13s/13-3s/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = "orange", lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v13s/13-3s/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = "orange", lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [33]:
combs <- expand.grid(tau = rep(c(0.10, 0.25, 0.3, 0.5, 1.0), each = 50), n = c(500, 750, 1000), quantile = c(0.05, 0.25, 0.5))

pdf("figs/11-3p-curve-1.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3p/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 3, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3p/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 3, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [34]:
# ! ---------------------------
# ! Spline with Ridge Penalty
# ! ---------------------------
pdf("figs/11-3p-curve-2.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v11s/11-3p/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = 4, lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v11s/11-3p/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = 4, lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2

In [35]:
pdf("figs/11-3p-curve-3.pdf", width = 17, height = 4)
par(mfrow = c(1, 3), mar = c(5, 5, 4, 2) + 0.4)

# 设置字体大小
cex_lab <- 3 # 坐标轴标签的大小
cex_axis <- 2 # 坐标轴数字的大小
cex_main <- 2 # 主标题的大小


n <- 750
SNRs <- c(0.10, 0.25, 0.5)
q <- 0.25

# 为每个kk值绘图
for (kk in 1:3) {
    beta.ave <- rep(0, 30)
    uus <- which(combs$tau == SNRs[kk] & combs$n == n & combs$quantile == q)
    uusample <- uus

    uu <- uusample[1]
    tmp <- paste0("Results-v13s/13-3p/Res_", uu, ".RDS")
    temp <- readRDS(tmp)

    beta.true <- matrix(temp$beta.all[,1], nrow = 6, byrow = TRUE)
    beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
    # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
    # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)

    # 绘制第一个plot
    if (kk == 1) {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "Coefficients", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    } else {
        plot(1:30, beta.true[1, ], type = 'o', col = 1, lwd = 2, ylim = c(-8, 8), xlab = "", ylab = "", main = paste("SNR =", SNRs[kk]), cex.lab = cex_lab, cex.axis = cex_axis, cex.main = cex_main)
    }
    
    lines(1:30, beta.uni[1, ], type = 'l', col = "orange", lty = 2)
    beta.ave <- beta.ave + beta.uni[1, ]
    # lines(1:30, beta.spline[1, ], type = 'o', col = 3, lty = 2)
    # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)

    # 绘制剩余的plots
    for (uu in uusample[-1]) {
        tmp <- paste0("Results-v13s/13-3p/Res_", uu, ".RDS")
        if (file.exists(tmp)) {
            temp <- readRDS(tmp)
        } else {
            cat(uu, ", ", sep = "")
        }
        beta.uni <- matrix(temp$beta.all[,3], nrow = 6, byrow = TRUE)
        # beta.spline <- matrix(temp$beta.all[,8], nrow = 6, byrow = TRUE)
        # beta.spline.ridge <- matrix(temp$beta.all[,9], nrow = 6, byrow = TRUE)
        lines(1:30, beta.uni[1,], type = 'l', col = "orange", lty = 2)
        beta.ave <- beta.ave + beta.uni[1, ]
        # lines(1:30, beta.spline[1,], type = 'o', col = 3, lty = 2)
        # lines(1:30, beta.spline.ridge[1, ], type = 'o', col = 4, lty = 2)
    }

    lines(1:30, beta.ave / length(uusample), type = 'o', col = 2, lty = 1, lwd = 2)
}
dev.off()

png 
  2